In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import TruncatedSVD
import random
import seaborn as sns
import os.path as path
import os
import matplotlib
import matplotlib.font_manager
import matplotlib.pyplot as plt # graphs plotting
from Bio import SeqIO # some BioPython that will come in handy
#matplotlib inline

from matplotlib import rc
# for Arial typefont
matplotlib.rcParams['font.family'] = 'Arial'

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import svm

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import classification_report, confusion_matrix 

from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
## for Palatino and other serif fonts use:
# rc('font',**{'family':'serif','serif':['Palatino']})
# matplotlib.rcParams['mathtext.fontset'] = 'cm'

## for LaTeX typefont
# matplotlib.rcParams['mathtext.fontset'] = 'stix'
# matplotlib.rcParams['font.family'] = 'STIXGeneral'

## for another LaTeX typefont
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

# rc('text', usetex = True)

print("done")

done


In [2]:
# Read the fasta-file and create a dictionary of its protein sequences
# input_file_name = "C:/Users/Sarwan/Desktop/4._Spike_sequences_aligned_aa_28-1015_gaps_removed_X_inserted.fas"
input_file_name = "D:/University/Ph.D/Advance Bioinformatics/Project/dataset/File5_gaps_and_unsolved_regions_removed_X_inserted_988aa.fasta"

# The fasta defline (name of a sequence) has the following format:
# Strain Name | Accession Number | Host Species | Virus Species) 
sequences_dictionary = {sequence.id : sequence.seq for sequence in SeqIO.parse(input_file_name,'fasta')}

print("done")

done


In [3]:
sequences_dictionary

{'human_mammal_gb:MT020781:21563-25384|Organism:Severe': Seq('YTNSFTRGVYYPDKVFRSSVLYSTQDLFLPFFSNVTWFHNPVLPFNDGVYFAST...DSF'),
 'human_mammal_gb:MW079427:21509-25330|Organism:Severe': Seq('YTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHNPVLPFNDGVYFAST...DSF'),
 'human_mammal_gb:MW547395:21530-25351|Organism:Severe': Seq('YTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHNPVLPFNDGVYFAST...DSF'),
 'bat_mammal_gb:GU190215:21391-25170|Organism:Bat': Seq('QTPSSRRGFYYFDDIFRSSIRVLTTGHFLPFNTNLTWYLNPNINFGDGVYFGLT...DSF'),
 'bat_mammal_gb:NC_014470:21391-25170|Organism:Bat': Seq('QTPSSRRGFYYFDDIFRSSIRVLTTGHFLPFNTNLTWYLNPNINFGDGVYFGLT...DSF'),
 'bat_mammal_gb:LC556375:21508-25215|Organism:Severe': Seq('YVSSRTRGVYYPDDAFRSSTNVLHEGFFLPFDSNVTWYSVATSPFGDGVYFSTI...ESF'),
 'bat_mammal_gb:KF569996|Organism:Rhinolophus': Seq('LVSSSKRGVYYPDDIYRSDVHHLVQDLFLPFNSNVVGLMNPIIPFKDGVYFAAT...DSF'),
 'bat_mammal_gb:KY417144|Organism:Bat': Seq('QFLSSHRGVYYPDDIFRSNVLHLVQDHFLPFDSNVTRFINPIIPFRDGIYFAAT...DSF'),
 'bat_mammal_gb:KC881005|Orga

In [3]:
# From the newly formed sequences_dictionary, we create 3 lists:
# a list of deflines,
# a list of sequences,
# and a list of target values

# We want to mark all sequences that belong to the viruses that can infect humans as 1 (i.e., target = 1), 
# all other sequences as 0 (i.e., target = 0)

human_virus_species_set =  {'Human_coronavirus_NL63', 'Betacoronavirus_1', 
                            'Human_coronavirus_HKU1', 'Severe_acute_respiratory_syndrome_related_coronavirus', 
                            'SARS_CoV_2', 'Human_coronavirus_229E', 'Middle_East_respiratory_syndrome_coronavirus'}

deflines = [entry for entry in sequences_dictionary.keys()]             # create a list of deflines
protein_sequences = [entry for entry in sequences_dictionary.values()]  # create a list of protein sequences 


# we assign 1 iff a virus species is one from the 7 human-infective

targets = [0]*len(deflines)
for i, defline in enumerate(deflines):
    for virus_species in human_virus_species_set:
        if virus_species in defline:
            targets[i] = 1   
            
print("done")

done


In [4]:
deflines

['human_mammal_gb:MT020781:21563-25384|Organism:Severe',
 'human_mammal_gb:MW079427:21509-25330|Organism:Severe',
 'human_mammal_gb:MW547395:21530-25351|Organism:Severe',
 'bat_mammal_gb:GU190215:21391-25170|Organism:Bat',
 'bat_mammal_gb:NC_014470:21391-25170|Organism:Bat',
 'bat_mammal_gb:LC556375:21508-25215|Organism:Severe',
 'bat_mammal_gb:KF569996|Organism:Rhinolophus',
 'bat_mammal_gb:KY417144|Organism:Bat',
 'bat_mammal_gb:KC881005|Organism:Bat',
 'bat_mammal_gb:KC881006|Organism:Bat',
 'bat_mammal_gb:KF367457|Organism:Bat',
 'bat_mammal_gb:MK211376|Organism:Coronavirus',
 'bat_mammal_gb:KY417151|Organism:Bat',
 'bat_mammal_gb:KY417152|Organism:Bat',
 'bat_mammal_gb:KY417146|Organism:Bat',
 'bat_mammal_gb:KT444582|Organism:SARS-like',
 'bat_mammal_gb:KY417150|Organism:Bat',
 'civet_mammal_gb:AY686864|Organism:SARS',
 'civet_mammal_gb:AY572038|Organism:SARS',
 'civet_mammal_gb:AY572035|Organism:SARS',
 'civet_mammal_gb:AY572034|Organism:Civet',
 'civet_mammal_gb:AY686863|Organis

In [4]:
cities = []
for i in range(0, len(deflines)):
    aaa = (deflines[i].split(":")[0])
    cities.append(aaa)
print(cities)

['human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'civet_mammal_gb', 'civet_mammal_gb', 'civet_mammal_gb', 'civet_mammal_gb', 'civet_mammal_gb', 'human_mammal_gb', 'swine_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'human_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb', 'bat_mammal_gb'

In [6]:
(np.unique(cities))

array(['bat_mammal_gb', 'belugaWhale_mammal_gb', 'camel_mammal_gb',
       'cat_mammal_gb', 'chicken_avian_gb', 'chimpanzee_mammal_gb',
       'civet_mammal_gb', 'duck_avian_gb', 'falcon_avian_gb',
       'goose_avian_gb', 'human_mammal_gb', 'pangolin_mammal_gb',
       'swine_mammal_gb'], dtype='<U21')

In [7]:
cities.count('human_mammal_gb')

957

In [9]:
(deflines)

['human_mammal_gb:MT020781:21563-25384|Organism:Severe',
 'human_mammal_gb:MW079427:21509-25330|Organism:Severe',
 'human_mammal_gb:MW547395:21530-25351|Organism:Severe',
 'bat_mammal_gb:GU190215:21391-25170|Organism:Bat',
 'bat_mammal_gb:NC_014470:21391-25170|Organism:Bat',
 'bat_mammal_gb:LC556375:21508-25215|Organism:Severe',
 'bat_mammal_gb:KF569996|Organism:Rhinolophus',
 'bat_mammal_gb:KY417144|Organism:Bat',
 'bat_mammal_gb:KC881005|Organism:Bat',
 'bat_mammal_gb:KC881006|Organism:Bat',
 'bat_mammal_gb:KF367457|Organism:Bat',
 'bat_mammal_gb:MK211376|Organism:Coronavirus',
 'bat_mammal_gb:KY417151|Organism:Bat',
 'bat_mammal_gb:KY417152|Organism:Bat',
 'bat_mammal_gb:KY417146|Organism:Bat',
 'bat_mammal_gb:KT444582|Organism:SARS-like',
 'bat_mammal_gb:KY417150|Organism:Bat',
 'civet_mammal_gb:AY686864|Organism:SARS',
 'civet_mammal_gb:AY572038|Organism:SARS',
 'civet_mammal_gb:AY572035|Organism:SARS',
 'civet_mammal_gb:AY572034|Organism:Civet',
 'civet_mammal_gb:AY686863|Organis

In [5]:
len(defline.split(":"))
defline.split(":")[0]

'bat_mammal_gb'

In [6]:
# We create a class fasta_sequence so that we would be able to use the sequence data easily 

class fasta_sequence:
    def __init__(self, defline, sequence, target, type_of_encoding = "onehot"):
        
        # we read the input data
        
        self.defline = defline
        self.sequence = sequence
        self.target = target
        
        # and create more descriptions of the input data
        
        # report the strain name (the 1st fiel of the defline)
#         self.strain_name = defline.split("|")[0]
        # report the accession number (the 2nd fiel of the defline)
        self.host_species = defline.split(":")[len(defline.split(":"))-1]        
#         # report the host species (the 3rd fiel of the defline)
#         self.host_species = defline.split("|")[2]    
#         # report the virus species (the 4th fiel of the defline)
#         self.virus_species = defline.split("|")[3]
        
        
# We convert a string with the alphabet = 'ABCDEFGHIJKLMNPQRSTUVWXYZ-' 
# into either a list mapping chars to integers (called integer encoding),
# or a sparce list. In the latter, each amino acid is represented as an one-hot vector of length 25, 
# where each position, except one, is set to 0.  E.g., alanine is encoded as 10000000000000000000, 
# cystine is encoded as 01000000000000000000
# See the full table above.
# Symbol '-' is encoded as a zero-vector.

        def encoding(sequence, type_of_encoding):

            # define universe of possible input values
            alphabet = 'ABCDEFGHIJKLMNPQRSTUVWXYZ-'
            # define a mapping of chars to integers
            char_to_int = dict((c, i) for i, c in enumerate(alphabet))


            # integer encoding
            integer_encoded = [char_to_int[char] for char in sequence]

            # one-hot encoding
            onehot_encoded = list()
            for value in integer_encoded:
                letter = [0 for _ in range(len(alphabet)-1)]
                if value != len(alphabet)-1:
                    letter[value] = 1
                onehot_encoded.append(letter)
            flat_list = [item for sublist in onehot_encoded for item in sublist]

            if type_of_encoding == "onehot":
                return flat_list
            else:
                return integer_encoded
            
        #  we use the encoding function to create a new attribute for the sequence -- its encoding        
        self.encoded = encoding(sequence, type_of_encoding)
        
print("done")

done


In [7]:
# we create a list of sequences as objects of the class fasta_sequence
# all sequences are encoded with one-hot encoding (it is the default option of the constructor of the class)
sequences = []
for i in range(0, len(deflines)):
    current_sequence = fasta_sequence(deflines[i],protein_sequences[i],targets[i])
    sequences.append(current_sequence)
    
print("done")

done


In [11]:
len(deflines)

2575

In [12]:
len(protein_sequences[0])

988

In [8]:
# for a list of sequences, returns a list of encoded sequences and a list of targets

def EncodeAndTarget(list_of_sequences):
    # encoding the sequences
    list_of_encoded_sequences = [entry.encoded for entry in list_of_sequences]
    # creating lists of targets
    list_of_targets = [entry.target for entry in list_of_sequences]
#     print(list_of_targets)
    return list_of_encoded_sequences, list_of_targets   
print("done")

done


In [9]:
encoded_sequences, targets = EncodeAndTarget(sequences)

In [10]:
# calculates explained variance for each of elements in number_of_components
def list_of_explained_variances(sequences, number_of_components, to_print = False):
    levels = []
    for i in number_of_components:
        svd = TruncatedSVD(n_components = i, n_iter = 7, random_state = 42)
        X = svd.fit_transform(sequences)
        explained_variances = svd.explained_variance_ratio_
        
        # the sum of all included explained variances 
        the_level = explained_variances.sum()
        if to_print == True:
            print(i, the_level)
        levels.append(the_level)
    return levels

In [11]:
number_of_components = list(range(2,101)) # + [150,200,300,500,1000]
# levels = list_of_explained_variances(encoded_sequences, number_of_components, to_print = True)

In [ ]:
levels = list_of_explained_variances(encoded_sequences, number_of_components, to_print = True)

2 0.34137724968470406
3 0.5793655781816749
4 0.7157461370837943
5 0.7717615156413522
6 0.8150293769266785
7 0.8430143211151244
8 0.8659284833628695
9 0.880550752164091
10 0.892215439850609
11 0.9000648637324746
12 0.906412320752717
13 0.9123548756393001
14 0.9180105373480324
15 0.9219386222934144
16 0.9255954339475563
17 0.9291217079717647
18 0.9324757872820237
19 0.9351893598193903
20 0.9376764878010099
21 0.9398625051364479
22 0.9418817419310478
23 0.9438150909060963
24 0.9455182964206034
25 0.9471776042894344
26 0.9487245868890496
27 0.9501290154582651
28 0.9514811587214907
29 0.9526931979214535
30 0.9538871922678659
31 0.9550351960485102
32 0.9559651858588982
33 0.9568523637930535


In [12]:
data = pd.DataFrame(encoded_sequences)

In [13]:
data.shape

(2575, 24700)

In [14]:
data_orig = data

In [50]:
svd = TruncatedSVD(n_components = 40, n_iter = 7, random_state = 42)
data_with_svd = svd.fit_transform(encoded_sequences)

# the command below is the same as
# explained_variances = np.var(X_50, axis=0) / np.var(encoded_sequences, axis=0).sum()
explained_variances = svd.explained_variance_ratio_

# the sum of all included explained variances 
the_level = explained_variances.sum()

In [51]:
data_with_svd.shape

(2575, 40)

In [26]:
len(cities)

2575

In [27]:
pd.DataFrame(data_with_svd).head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,16.537420,24.121557,-11.113669,-1.647128,-0.772342,-0.151101,-0.096523,-1.448130,-0.006159,-0.039778,...,0.024309,0.006666,0.023379,0.016306,0.000187,0.006700,-0.004204,-0.001828,-0.025308,0.053750
1,10.870367,16.942443,-9.831833,-2.428878,-1.192669,-0.470540,0.245992,-3.162796,0.210265,-0.535568,...,-0.323459,0.469482,-0.759465,0.187131,-0.602898,-0.201110,0.318672,-1.363172,1.310215,-1.352417
2,16.596025,18.946177,-10.190149,-2.137447,-0.643434,-0.157182,0.604134,-0.736895,0.374845,0.581264,...,0.063881,-0.021667,-0.082374,-0.235393,-0.028872,0.001039,-0.112528,-0.154934,0.121638,-0.645862
3,14.469282,18.904534,-6.805299,-0.525098,0.496996,0.079308,0.049833,10.080149,-0.231696,-0.276268,...,0.324932,0.615505,0.185744,0.842856,0.187546,-0.446752,-0.638543,0.769085,-0.859083,5.804149
4,14.469282,18.904534,-6.805299,-0.525098,0.496996,0.079308,0.049833,10.080149,-0.231696,-0.276268,...,0.324932,0.615505,0.185744,0.842856,0.187546,-0.446752,-0.638543,0.769085,-0.859083,5.804149


In [15]:
# data = pd.DataFrame(data_with_svd)
data = pd.DataFrame(data_orig)

In [16]:
final_labels_temp = [] 
for entry in cities:
    if entry == 'human_mammal_gb':
        final_labels_temp.append(1)
    elif entry == 'camel_mammal_gb':
        final_labels_temp.append(2)
    elif entry == 'cat_mammal_gb':
        final_labels_temp.append(3)
    elif entry == 'chicken_avian_gb':
        final_labels_temp.append(4)
    elif entry == 'bat_mammal_gb':
        final_labels_temp.append(5)
    elif entry == 'swine_mammal_gb':
        final_labels_temp.append(6)
    else:
        final_labels_temp.append(0)

In [17]:
sum(final_labels_temp)

8509

In [19]:
data_labels = final_labels_temp

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    data,data_labels,
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((1802, 24700), (773, 24700))

# LASSO Ridge regression for feature selection

In [37]:
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

StandardScaler()

In [38]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [39]:
sel_.get_support()

array([False, False, False, ..., False, False, False])

In [40]:
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 24700
selected features: 618
features with coefficients shrank to zero: 172104


In [41]:
X_train_selected = sel_.transform(X_train.fillna(0))
X_test_selected = sel_.transform(X_test.fillna(0))
X_train_selected.shape, X_test_selected.shape

((1802, 618), (773, 618))

In [42]:
X_train = X_train_selected
X_test = X_test_selected

# X_train = X_train_encoder
# X_test = X_test_encoder


# Classifier Functions

In [27]:
# In[4]
def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc
    
    
  check = pd.DataFrame(roc_auc_dict.items())
  return mean(check)

# In[5]
##########################  SVM Classifier  ################################
def svm_fun(X_train,y_train,X_test,y_test):
    #Create a svm Classifier
    clf = svm.SVC(kernel='linear') # Linear Kernel

    #Train the model using the training sets
    clf.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    
    svm_acc = metrics.accuracy_score(y_test, y_pred)
#     print("SVM Accuracy:",svm_acc)
    
    svm_prec = metrics.precision_score(y_test, y_pred,average='weighted')
#     print("SVM Precision:",svm_prec)
    
    svm_recall = metrics.recall_score(y_test, y_pred,average='weighted')
#     print("SVM Recall:",svm_recall)

    svm_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
#     print("SVM F1 Weighted:",svm_f1_weighted)
    
    svm_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
#     print("SVM F1 macro:",svm_f1_macro)
    
    svm_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
#     print("SVM F1 micro:",svm_f1_micro)
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix SVM : \n", confuse)
    ######################## Compute ROC curve and ROC area for each class ################
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
#    print(macro_roc_auc_ovo[1])
    check = [svm_acc,svm_prec,svm_recall,svm_f1_weighted,svm_f1_macro,svm_f1_micro,macro_roc_auc_ovo[1]]
    return(check)
    


# In[5]
##########################  NB Classifier  ################################
def gaus_nb_fun(X_train,y_train,X_test,y_test):
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)


    NB_acc = metrics.accuracy_score(y_test, y_pred)
#     print("Gaussian NB Accuracy:",NB_acc)

    NB_prec = metrics.precision_score(y_test, y_pred,average='weighted')
#     print("Gaussian NB Precision:",NB_prec)
    
    NB_recall = metrics.recall_score(y_test, y_pred,average='weighted')
#     print("Gaussian NB Recall:",NB_recall)
    
    NB_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
#     print("Gaussian NB F1 weighted:",NB_f1_weighted)
    
    NB_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
#     print("Gaussian NB F1 macro:",NB_f1_macro)
    
    NB_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
#     print("Gaussian NB F1 micro:",NB_f1_micro)
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix NB : \n", confuse)
    ######################## Compute ROC curve and ROC area for each class ################
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    check = [NB_acc,NB_prec,NB_recall,NB_f1_weighted,NB_f1_macro,NB_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

# In[5]
##########################  MLP Classifier  ################################
def mlp_fun(X_train,y_train,X_test,y_test):
    # Feature scaling
    scaler = StandardScaler()  
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)  
    X_test_2 = scaler.transform(X_test)


    # Finally for the MLP- Multilayer Perceptron
    mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  
    mlp.fit(X_train, y_train)


    y_pred = mlp.predict(X_test_2)
    
    MLP_acc = metrics.accuracy_score(y_test, y_pred)
#     print("MLP Accuracy:",MLP_acc)
    
    MLP_prec = metrics.precision_score(y_test, y_pred,average='weighted')
#     print("MLP Precision:",MLP_prec)
    
    MLP_recall = metrics.recall_score(y_test, y_pred,average='weighted')
#     print("MLP Recall:",MLP_recall)
    
    MLP_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
#     print("MLP F1:",MLP_f1_weighted)
    
    MLP_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
#     print("MLP F1:",MLP_f1_macro)
    
    MLP_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
#     print("MLP F1:",MLP_f1_micro)
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix MLP : \n", confuse)
    ######################## Compute ROC curve and ROC area for each class ################
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    
    check = [MLP_acc,MLP_prec,MLP_recall,MLP_f1_weighted,MLP_f1_macro,MLP_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

# In[5]
##########################  knn Classifier  ################################
def knn_fun(X_train,y_train,X_test,y_test):
    classifier = KNeighborsClassifier(n_neighbors=5)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    knn_acc = metrics.accuracy_score(y_test, y_pred)
#     print("Knn Accuracy:",knn_acc)
    
    knn_prec = metrics.precision_score(y_test, y_pred,average='weighted')
#     print("Knn Precision:",knn_prec)
    
    knn_recall = metrics.recall_score(y_test, y_pred,average='weighted')
#     print("Knn Recall:",knn_recall)
    
    knn_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
#     print("Knn F1 weighted:",knn_f1_weighted)
    
    knn_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
#     print("Knn F1 macro:",knn_f1_macro)
    
    knn_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
#     print("Knn F1 micro:",knn_f1_micro)
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix KNN : \n", confuse)
    ######################## Compute ROC curve and ROC area for each class ################
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    
    check = [knn_acc,knn_prec,knn_recall,knn_f1_weighted,knn_f1_macro,knn_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

# In[5]
##########################  Random Forest Classifier  ################################
def rf_fun(X_train,y_train,X_test,y_test):
    # Import the model we are using
    from sklearn.ensemble import RandomForestClassifier
    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(n_estimators = 100)
    # Train the model on training data
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    fr_acc = metrics.accuracy_score(y_test, y_pred)
#     print("Random Forest Accuracy:",fr_acc)
    
    fr_prec = metrics.precision_score(y_test, y_pred,average='weighted')
#     print("Random Forest Precision:",fr_prec)
    
    fr_recall = metrics.recall_score(y_test, y_pred,average='weighted')
#     print("Random Forest Recall:",fr_recall)
    
    fr_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
#     print("Random Forest F1 weighted:",fr_f1_weighted)
    
    fr_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
#     print("Random Forest F1 macro:",fr_f1_macro)
    
    fr_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
#     print("Random Forest F1 micro:",fr_f1_micro)
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix RF : \n", confuse)
    ######################## Compute ROC curve and ROC area for each class ################
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    
    check = [fr_acc,fr_prec,fr_recall,fr_f1_weighted,fr_f1_macro,fr_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

# In[5]
    ##########################  Logistic Regression Classifier  ################################
def lr_fun(X_train,y_train,X_test,y_test):

    model = LogisticRegression(solver='liblinear', random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    LR_acc = metrics.accuracy_score(y_test, y_pred)
#     print("Logistic Regression Accuracy:",LR_acc)
    
    LR_prec = metrics.precision_score(y_test, y_pred,average='weighted')
#     print("Logistic Regression Precision:",LR_prec)
    
    LR_recall = metrics.recall_score(y_test, y_pred,average='weighted')
#     print("Logistic Regression Recall:",LR_recall)
    
    LR_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
#     print("Logistic Regression F1 weighted:",LR_f1_weighted)
    
    LR_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
#     print("Logistic Regression F1 macro:",LR_f1_macro)
    
    LR_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
#     print("Logistic Regression F1 micro:",LR_f1_micro)
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix LR : \n", confuse)
    ######################## Compute ROC curve and ROC area for each class ################
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    
    check = [LR_acc,LR_prec,LR_recall,LR_f1_weighted,LR_f1_macro,LR_f1_micro,macro_roc_auc_ovo[1]]
    return(check)


# K Fold Cross Validation

In [45]:
data_selected = sel_.transform(data.fillna(0)) # For Lasso and Ridge
# data_selected = data_with_svd #for PCA
data_selected.shape

(2575, 618)

In [46]:
X = data_selected
y =  np.array(data_labels)

rkf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=None)
# X is the feature set and y is the target
counter = 1
# print("Accuracy   Precision   Recall   F1 (weighted)   F1 (Macro)   F1 (Micro)   ROC AUC")
svm_table = []
gauu_nb_table = []
mlp_table = []
knn_table = []
rf_table = []
lr_table = []

for train_index, test_index in rkf.split(X):
     counter = counter+1
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
    
     svm_return = svm_fun(X_train,y_train,X_test,y_test)
     gauu_nb_return = gaus_nb_fun(X_train,y_train,X_test,y_test)
     mlp_return = mlp_fun(X_train,y_train,X_test,y_test)
     knn_return = knn_fun(X_train,y_train,X_test,y_test)
     rf_return = rf_fun(X_train,y_train,X_test,y_test)
     lr_return = lr_fun(X_train,y_train,X_test,y_test)
    
     svm_table.append(svm_return)
     gauu_nb_table.append(gauu_nb_return)
     mlp_table.append(mlp_return)
     knn_table.append(knn_return)
     rf_table.append(rf_return)
     lr_table.append(lr_return)
     
svm_table_final = DataFrame(svm_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
gauu_nb_table_final = DataFrame(gauu_nb_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
mlp_table_final = DataFrame(mlp_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
knn_table_final = DataFrame(knn_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
rf_table_final = DataFrame(rf_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
lr_table_final = DataFrame(lr_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])


Confusion Matrix SVM : 
 [[ 2  0  0  0  0  1  0]
 [ 0 93 17  0  0  0  0]
 [ 0  0 27  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 26  0  0]
 [ 0  0  0  0  0 13  1]
 [ 0  0  0  0  0  0 76]]
Confusion Matrix NB : 
 [[ 2  0  0  0  0  1  0]
 [ 0 89 21  0  0  0  0]
 [ 0  0 27  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 1  0  0  0 25  0  0]
 [ 0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0 76]]
Confusion Matrix MLP : 
 [[ 2  0  0  0  0  1  0]
 [ 0 92 18  0  0  0  0]
 [ 0  0 27  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 26  0  0]
 [ 0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0 76]]
Confusion Matrix KNN : 
 [[  2   1   0   0   0   0   0]
 [  0 102   8   0   0   0   0]
 [  0  10  17   0   0   0   0]
 [  0   0   0   2   0   0   0]
 [  0   0   0   0  26   0   0]
 [  0   0   1   0   0  11   2]
 [  0   0   0   0   0   0  76]]
Confusion Matrix RF : 
 [[ 2  0  0  0  0  1  0]
 [ 0 92 18  0  0  0  0]
 [ 0  0 27  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 26  0  0]
 [ 0  0  0  0  0 13  1]
 [ 0  0 

C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix SVM : 
 [[ 0  0  0  0  0  0  0]
 [ 1 73 14  0  0  0  0]
 [ 0  2 27  0  0  0  0]
 [ 0  1  0 11  0  0  0]
 [ 0  0  0  0 29  0  0]
 [ 0  0  0  0  0 18  0]
 [ 0  0  0  0  0  0 82]]
Confusion Matrix NB : 
 [[ 0  0  0  0  0  0  0]
 [ 1 67 20  0  0  0  0]
 [ 0  2 27  0  0  0  0]
 [ 0  1  0 11  0  0  0]
 [ 0  0  0  0 29  0  0]
 [ 0  0  0  0  0 18  0]
 [ 0  0  0  0  0  0 82]]


C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix MLP : 
 [[ 0  0  0  0  0  0  0]
 [ 1 73 14  0  0  0  0]
 [ 0  2 27  0  0  0  0]
 [ 0  1  0 11  0  0  0]
 [ 0  0  0  0 29  0  0]
 [ 0  0  0  0  0 18  0]
 [ 0  0  0  0  0  0 82]]


...

Confusion Matrix RF : 
 [[ 0  0  0  0  0  0  0]
 [ 1 73 14  0  0  0  0]
 [ 0  3 26  0  0  0  0]
 [ 0  1  0 11  0  0  0]
 [ 0  0  0  0 29  0  0]
 [ 0  0  0  0  0 18  0]
 [ 0  0  0  0  0  0 82]]
Confusion Matrix LR : 
 [[74 14  0  0  0  0]
 [ 3 26  0  0  0  0]
 [ 1  0 11  0  0  0]
 [ 0  0  0 29  0  0]
 [ 0  0  0  0 18  0]
 [ 0  0  0  0  0 82]]
Confusion Matrix SVM : 
 [[ 3  0  0  0  1  0  0]
 [ 0 80  6  0  0  0  0]
 [ 0  2 24  0  0  0  0]
 [ 0  0  0  5  0  0  0]
 [ 0  0  0  0 34  0  0]
 [ 0  1  0  0  0 14  0]
 [ 0  0  0  0  0  0 87]]
Confusion Matrix NB : 
 [[ 3  0  0  0  1  0  0]
 [ 0 77  9  0  0  0  0]
 [ 0  1 25  0  0  0  0]
 [ 0  0  0  5  0  0  0]
 [ 0  0  0  0 34  0  0]
 [ 0  1  0  0  0 14  0]
 [ 0  0  0  0  0  0 87]]
Confusion Matrix MLP : 
 [[ 3  0  0  0  1  0  0]
 [ 0 80  6  0  0  0  0]
 [ 0  2 24  0  0  0  0]
 [ 0  0  0  5  0  0  0]
 [ 0  0  0  0 34  0  0]
 [ 0  1  0  0  0 14  0]
 [ 0  0  0  0  0  0 87]]
Confusion Matrix KNN : 
 [[ 2  0  1  0  1  0  0]
 [ 0 85  1  0  0  0  0]
 [

C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix KNN : 
 [[ 0  0  1  0  0  0  0]
 [ 0 91  1  0  0  0  0]
 [ 0 10 14  0  0  0  0]
 [ 0  0  0  7  0  0  0]
 [ 0  0  0  0 31  0  0]
 [ 0  0  1  0  0 15  1]
 [ 0  0  0  0  0  0 85]]
Confusion Matrix RF : 
 [[ 1  0  0  0  0  0  0]
 [ 0 87  5  0  0  0  0]
 [ 0  3 21  0  0  0  0]
 [ 0  0  0  7  0  0  0]
 [ 0  0  0  0 31  0  0]
 [ 0  0  1  0  0 16  0]
 [ 0  0  0  0  0  0 85]]
Confusion Matrix LR : 
 [[ 1  0  0  0  0  0  0]
 [ 0 90  2  0  0  0  0]
 [ 0 10 14  0  0  0  0]
 [ 0  0  0  7  0  0  0]
 [ 0  0  0  0 31  0  0]
 [ 0  0  1  0  0 16  0]
 [ 0  0  0  0  0  0 85]]
Confusion Matrix SVM : 
 [[ 1  0  0  0  0  0  0]
 [ 0 85  6  0  0  0  0]
 [ 0  2 32  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 45  0  0]
 [ 0  0  0  0  0 18  1]
 [ 0  0  0  0  0  0 65]]
Confusion Matrix NB : 
 [[ 1  0  0  0  0  0  0]
 [ 0 82  9  0  0  0  0]
 [ 0  0 34  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 45  0  0]
 [ 0  0  0  0  0 19  0]
 [ 0  0  0  0  0  0 65]]
Confusion Matrix MLP : 
 [[ 1  0 

C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix KNN : 
 [[ 0  0  1  0  0  0  0]
 [ 0 89  2  0  0  0  0]
 [ 0 15 19  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 45  0  0]
 [ 0  0  1  0  0 16  2]
 [ 0  0  0  0  0  1 64]]
Confusion Matrix RF : 
 [[ 1  0  0  0  0  0  0]
 [ 0 86  5  0  0  0  0]
 [ 0  1 33  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 45  0  0]
 [ 0  0  0  0  0 18  1]
 [ 0  0  0  0  0  0 65]]
Confusion Matrix LR : 
 [[ 1  0  0  0  0  0  0]
 [ 0 91  0  0  0  0  0]
 [ 0 17 17  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 45  0  0]
 [ 0  0  0  0  0 18  1]
 [ 0  0  0  0  0  0 65]]


In [47]:
#taking average of all k-fold performance values
final_mean_mat = []

final_mean_mat.append(np.transpose((list(svm_table_final.mean()))))
final_mean_mat.append(np.transpose((list(gauu_nb_table_final.mean()))))
final_mean_mat.append(np.transpose((list(mlp_table_final.mean()))))
final_mean_mat.append(np.transpose((list(knn_table_final.mean()))))
final_mean_mat.append(np.transpose((list(rf_table_final.mean()))))
final_mean_mat.append(np.transpose((list(lr_table_final.mean()))))

final_avg_mat = DataFrame(final_mean_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["SVM","NB","MLP","KNN","RF","LR"])

final_avg_mat

,Accuracy,Precision,Recall,F1 (weighted),F1 (Macro),F1 (Micro),ROC AUC
SVM,0.947208,0.958134,0.947208,0.949642,0.928848,0.947208,0.970125
NB,0.935959,0.955885,0.935959,0.939974,0.920035,0.935959,0.970342
MLP,0.938648,0.947055,0.938648,0.937744,0.912407,0.938648,0.958931
KNN,0.926219,0.924260,0.926219,0.920220,0.856020,0.926219,0.915981
RF,0.950703,0.960679,0.950703,0.953041,0.938204,0.950703,0.975410
LR,0.935537,0.944578,0.935537,0.932470,0.923726,0.935537,0.950793


In [48]:
#taking variance of all k-fold performance values
final_var_mat = []

final_var_mat.append(np.transpose((list(svm_table_final.var()))))
final_var_mat.append(np.transpose((list(gauu_nb_table_final.var()))))
final_var_mat.append(np.transpose((list(mlp_table_final.var()))))
final_var_mat.append(np.transpose((list(knn_table_final.var()))))
final_var_mat.append(np.transpose((list(rf_table_final.var()))))
final_var_mat.append(np.transpose((list(lr_table_final.var()))))

final_var_mat = DataFrame(final_var_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["SVM","NB","MLP","KNN","RF","LR"])
final_var_mat

,Accuracy,Precision,Recall,F1 (weighted),F1 (Macro),F1 (Micro),ROC AUC
SVM,0.000216,0.000075,0.000216,0.000173,0.002358,0.000216,0.000135
NB,0.000508,0.000192,0.000508,0.000426,0.002994,0.000508,0.000278
MLP,0.000119,0.000094,0.000119,0.000143,0.001803,0.000119,0.000110
KNN,0.000224,0.000227,0.000224,0.000258,0.003761,0.000224,0.000791
RF,0.000269,0.000101,0.000269,0.000219,0.002681,0.000269,0.000133
LR,0.000077,0.000071,0.000077,0.000141,0.000327,0.000077,0.000278


In [23]:
# from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten, Dense, Reshape

from keras.layers import Input,Dense


# Autoencoder Starts Here

In [110]:
X_train, X_test, y_train, y_test = train_test_split(
    data_orig,data_labels,
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((1802, 24700), (773, 24700))

In [24]:
# reduce to 20 features
encoding_dim = 200

orig_shape = 24700
input_df = Input(shape=(orig_shape,))
encoded = Dense(encoding_dim, activation='relu')(input_df)
decoded = Dense(orig_shape, activation='sigmoid')(encoded)

# encoder
autoencoder = Model(input_df, decoded)

# intermediate result
encoder = Model(input_df, encoded)

autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

autoencoder.fit(X_train, X_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Epoch 1/20
8/8 [==============================] - 4s 472ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 2/20
8/8 [==============================] - 2s 240ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 3/20
8/8 [==============================] - 2s 258ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 4/20
8/8 [==============================] - 2s 265ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 5/20
8/8 [==============================] - 2s 226ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 6/20
8/8 [==============================] - 2s 229ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 7/20
8/8 [==============================] - 2s 232ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 8/20
8/8 [==============================] - 2s 241ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 9/20
8/8 [==============================] - 3s 377ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 10/20
8/8 [==============================] - 2s 234ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 11/20
8/8 [==

In [25]:
data_encoder = encoder.predict(data)

In [26]:
data_encoder.shape

(2575, 200)

# K Fold Cross Validation

In [33]:
from sklearn.model_selection import KFold 
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import confusion_matrix
from numpy import mean
from sklearn.metrics import roc_auc_score
from pandas import DataFrame


data_selected = data_encoder

X = data_selected
y =  np.array(data_labels)

rkf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=None)
# X is the feature set and y is the target
counter = 1
# print("Accuracy   Precision   Recall   F1 (weighted)   F1 (Macro)   F1 (Micro)   ROC AUC")
svm_table = []
gauu_nb_table = []
mlp_table = []
knn_table = []
rf_table = []
lr_table = []

for train_index, test_index in rkf.split(X):
     counter = counter+1
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
    
     svm_return = svm_fun(X_train,y_train,X_test,y_test)
     gauu_nb_return = gaus_nb_fun(X_train,y_train,X_test,y_test)
     mlp_return = mlp_fun(X_train,y_train,X_test,y_test)
     knn_return = knn_fun(X_train,y_train,X_test,y_test)
     rf_return = rf_fun(X_train,y_train,X_test,y_test)
     lr_return = lr_fun(X_train,y_train,X_test,y_test)
    
     svm_table.append(svm_return)
     gauu_nb_table.append(gauu_nb_return)
     mlp_table.append(mlp_return)
     knn_table.append(knn_return)
     rf_table.append(rf_return)
     lr_table.append(lr_return)
     
svm_table_final = DataFrame(svm_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
gauu_nb_table_final = DataFrame(gauu_nb_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
mlp_table_final = DataFrame(mlp_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
knn_table_final = DataFrame(knn_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
rf_table_final = DataFrame(rf_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
lr_table_final = DataFrame(lr_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])


C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix SVM : 
 [[  0   0   0   0   1   0   0]
 [  0 102   0   0   0   0   0]
 [  0  18  14   0   0   0   0]
 [  0   0   0   2   0   0   0]
 [  0   0   0   0  34   0   0]
 [  0   0   0   0   0  13   0]
 [  0   0   0   0   0   0  74]]
Confusion Matrix NB : 
 [[ 0  0  0  0  1  0  0]
 [13 65 22  0  0  1  1]
 [ 0  1 30  0  0  1  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 34  0  0]
 [ 4  0  0  0  0  7  2]
 [ 1  0  0  1  0  7 65]]
Confusion Matrix MLP : 
 [[ 0  0  0  0  1  0  0]
 [ 1 89 12  0  0  0  0]
 [ 0  1 31  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0 34  0  0]
 [ 0  0  0  0  0 13  0]
 [ 0  0  0  0  0  0 74]]
Confusion Matrix KNN : 
 [[  0   0   0   0   1   0   0]
 [  1 100   1   0   0   0   0]
 [  0  13  19   0   0   0   0]
 [  0   0   0   2   0   0   0]
 [  0   0   0   0  34   0   0]
 [  0   0   0   0   0  13   0]
 [  0   0   0   0   0   0  74]]
Confusion Matrix RF : 
 [[ 0  0  0  0  1  0  0]
 [ 1 93  8  0  0  0  0]
 [ 0  2 30  0  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0

C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix SVM : 
 [[  1   0   0   0   0   0   0]
 [  0 101   0   0   0   0   0]
 [  0  20   8   0   0   0   0]
 [  0   0   0   2   0   0   2]
 [  0   0   0   0  38   0   0]
 [  0   0   0   0   0  22   1]
 [  0   0   0   0   0   0  63]]
Confusion Matrix NB : 
 [[ 1  0  0  0  0  0  0]
 [14 65 19  0  0  2  1]
 [ 0  1 27  0  0  0  0]
 [ 0  0  0  4  0  0  0]
 [ 0  0  0  0 37  1  0]
 [ 0  0  0  0  0 15  8]
 [ 1  0  0  1  0  5 56]]
Confusion Matrix MLP : 
 [[ 1  0  0  0  0  0  0]
 [ 0 94  7  0  0  0  0]
 [ 0  3 25  0  0  0  0]
 [ 0  0  0  4  0  0  0]
 [ 0  0  0  0 38  0  0]
 [ 0  0  1  0  0 22  0]
 [ 0  0  0  0  0  0 63]]
Confusion Matrix KNN : 
 [[ 0  0  0  0  1  0  0]
 [ 0 95  6  0  0  0  0]
 [ 0  2 26  0  0  0  0]
 [ 0  0  0  4  0  0  0]
 [ 0  0  0  0 38  0  0]
 [ 0  0  0  0  0 22  1]
 [ 0  0  0  0  0  0 63]]


C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix MLP : 
 [[ 0  1  0  0  0  0  0]
 [ 0 79  8  0  0  0  0]
 [ 0  3 26  0  0  0  0]
 [ 0  0  1 10  0  0  0]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0 77]]
Confusion Matrix KNN : 
 [[ 0  1  0  0  0  0  0]
 [ 0 79  8  0  0  0  0]
 [ 0  4 25  0  0  0  0]
 [ 0  1  0 10  0  0  0]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0 77]]
Confusion Matrix RF : 
 [[ 1  0  0  0  0  0  0]
 [ 0 80  7  0  0  0  0]
 [ 0  6 23  0  0  0  0]
 [ 0  1  0 10  0  0  0]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0 77]]
Confusion Matrix LR : 
 [[ 0  1  0  0  0  0  0]
 [ 0 84  3  0  0  0  0]
 [ 0 19 10  0  0  0  0]
 [ 0  1  0  8  0  0  2]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0 77]]


C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix SVM : 
 [[  0   1   0   0   0   0   0]
 [  0 102   0   0   0   0   0]
 [  0  15   7   0   0   0   0]
 [  0   0   0   2   0   0   1]
 [  0   0   0   0  35   0   0]
 [  0   0   0   0   0  15   0]
 [  0   0   0   0   0   0  79]]
Confusion Matrix NB : 
 [[ 1  0  0  0  0  0  0]
 [11 56 31  0  0  4  0]
 [ 0  2 20  0  0  0  0]
 [ 0  0  0  3  0  0  0]
 [ 1  0  0  0 33  1  0]
 [ 2  0  0  0  0  8  5]
 [ 1  0  0  2  0  3 73]]
Confusion Matrix MLP : 
 [[ 1  0  0  0  0  0  0]
 [ 0 92 10  0  0  0  0]
 [ 0  3 17  0  0  2  0]
 [ 0  0  0  3  0  0  0]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 15  0]
 [ 0  0  0  0  0  0 79]]
Confusion Matrix KNN : 
 [[ 1  0  0  0  0  0  0]
 [ 0 93  9  0  0  0  0]
 [ 0  8 14  0  0  0  0]
 [ 0  0  0  3  0  0  0]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 15  0]
 [ 0  0  0  0  0  0 79]]
Confusion Matrix RF : 
 [[ 1  0  0  0  0  0  0]
 [ 0 92 10  0  0  0  0]
 [ 0  4 18  0  0  0  0]
 [ 0  0  0  3  0  0  0]
 [ 0  0  0  0 35  0  0]
 [ 0  0  0  0  0 15  0]
 [ 0  0 

C:\Users\Sarwan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
#taking average of all k-fold performance values
final_mean_mat = []

final_mean_mat.append(np.transpose((list(svm_table_final.mean()))))
final_mean_mat.append(np.transpose((list(gauu_nb_table_final.mean()))))
final_mean_mat.append(np.transpose((list(mlp_table_final.mean()))))
final_mean_mat.append(np.transpose((list(knn_table_final.mean()))))
final_mean_mat.append(np.transpose((list(rf_table_final.mean()))))
final_mean_mat.append(np.transpose((list(lr_table_final.mean()))))

final_avg_mat = DataFrame(final_mean_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["SVM","NB","MLP","KNN","RF","LR"])

final_avg_mat

,Accuracy,Precision,Recall,F1 (weighted),F1 (Macro),F1 (Micro),ROC AUC
SVM,0.916483,0.923709,0.916483,0.899137,0.816984,0.916483,0.885681
NB,0.781355,0.873768,0.781355,0.801196,0.692947,0.781355,0.892647
MLP,0.934734,0.937108,0.934734,0.931545,0.890253,0.934734,0.939978
KNN,0.934357,0.941451,0.934357,0.934614,0.862806,0.934357,0.929602
RF,0.952611,0.957542,0.952611,0.953837,0.940218,0.952611,0.968591
LR,0.909103,0.907462,0.909103,0.895289,0.796885,0.909103,0.876568


In [35]:
#taking variance of all k-fold performance values
final_var_mat = []

final_var_mat.append(np.transpose((list(svm_table_final.var()))))
final_var_mat.append(np.transpose((list(gauu_nb_table_final.var()))))
final_var_mat.append(np.transpose((list(mlp_table_final.var()))))
final_var_mat.append(np.transpose((list(knn_table_final.var()))))
final_var_mat.append(np.transpose((list(rf_table_final.var()))))
final_var_mat.append(np.transpose((list(lr_table_final.var()))))

final_var_mat = DataFrame(final_var_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["SVM","NB","MLP","KNN","RF","LR"])
final_var_mat

,Accuracy,Precision,Recall,F1 (weighted),F1 (Macro),F1 (Micro),ROC AUC
SVM,0.000371,0.000367,0.000371,0.000625,0.003075,0.000371,0.000650
NB,0.000418,0.000306,0.000418,0.000434,0.000782,0.000418,0.000471
MLP,0.000490,0.000770,0.000490,0.000762,0.002734,0.000490,0.000837
KNN,0.000302,0.000135,0.000302,0.000267,0.004050,0.000302,0.001078
RF,0.000081,0.000058,0.000081,0.000074,0.001756,0.000081,0.000469
LR,0.000284,0.000313,0.000284,0.000506,0.002173,0.000284,0.000402
